In [1]:
import duckdb
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme()

%load_ext sql

%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///:memory:

In [2]:
top_user_win_rate_percentile = 0.85

In [3]:
%%sql

DROP TYPE IF EXISTS expansion;
CREATE TYPE expansion AS ENUM ('KTK', 'LCI', 'WOE', 'LTR', 'MOM', 'SIR', 'ONE', 'BRO', 'DMU', 'HBG', 'SNC', 'NEO', 'VOW', 'MID', 'AFR', 'STX', 'KHM');
DROP TYPE IF EXISTS event_type;
CREATE TYPE event_type AS ENUM ('PremierDraft', 'TradDraft', 'Sealed', 'TradSealed');
DROP TYPE IF EXISTS rank;
CREATE TYPE rank AS ENUM ('silver', 'bronze', 'None', 'gold', 'platinum', 'diamond', 'mythic');
DROP TYPE IF EXISTS color;
CREATE TYPE color AS ENUM ('RG', 'UBG', 'WUB', 'UBR', 'WUBG', 'BG', 'WG', 'WBG', 'WUBRG', 'UR', 'UB', 'WBR', 'WRG', 'None', 'BR', 'WUBR', 'G', 'WR', 'W', 'WU', 'WB', 'WUR', 'WUG', 'WBRG', 'BRG', 'U', 'URG', 'UBRG', 'UG', 'B', 'R', 'WURG');

,Success


In [12]:
%%sql

DROP TABLE IF EXISTS draft;

# https://www.17lands.com/history/<draft_id>/<match_number - 1>/<game_number - 1>
CREATE TABLE draft (
    expansion expansion NOT NULL,
    event_type event_type NOT NULL,
    draft_id VARCHAR NOT NULL,
    draft_time TIMESTAMP_S NOT NULL,
    rank rank,
    
    event_match_wins TINYINT NOT NULL,
    event_match_losses TINYINT NOT NULL,
    
    pack_number TINYINT NOT NULL,
    pick_number TINYINT NOT NULL,

    pick VARCHAR NOT NULL,

    # the number of games the user played with the picked card in their maindeck, divided by the total number of games the user played (with that draft pool).
    pick_maindeck_rate DOUBLE NOT NULL,
    # the number of games the user played with the picked card in their sideboard, divided by the total number of games the user played (with that draft pool).
    pick_sideboard_in_rate DOUBLE NOT NULL,
    
    user_n_games_bucket INTEGER NOT NULL,
    user_game_win_rate_bucket DOUBLE
)

,Success


In [13]:
%%sql

INSERT INTO
    draft
BY NAME
SELECT
    expansion,
    event_type,
    draft_id,
    draft_time,
    rank,
    event_match_wins,
    event_match_losses,
    pack_number,
    pick_number,
    pick,
    pick_maindeck_rate,
    pick_sideboard_in_rate,
    user_n_games_bucket,
    user_game_win_rate_bucket,
FROM
    'https://17lands-public.s3.amazonaws.com/analysis_data/draft_data/draft_data_public.LCI.PremierDraft.csv.gz'

,Success


In [35]:
%%sql

DROP INDEX IF EXISTS draft_draft_pack_pick;
CREATE UNIQUE INDEX draft_draft_pack_pick ON draft (draft_id, pack_number, pick_number);

DROP INDEX IF EXISTS draft_rank;
CREATE INDEX draft_rank ON draft (rank);

DROP INDEX IF EXISTS draft_user_n_games_bucket;
CREATE INDEX draft_user_n_games_bucket ON draft (user_n_games_bucket);

DROP INDEX IF EXISTS draft_user_win_rate_bucket;
CREATE INDEX draft_user_win_rate_bucket ON draft (user_game_win_rate_bucket);


,Success


In [43]:
%sql SELECT *, (pick_maindeck_rate + pick_sideboard_in_rate) AS total FROM draft WHERE total <> 1 AND total <> 0 LIMIT 10;

,expansion,event_type,draft_id,draft_time,rank,event_match_wins,event_match_losses,pack_number,pick_number,pick,pick_maindeck_rate,pick_sideboard_in_rate,user_n_games_bucket,user_game_win_rate_bucket,total
0,LCI,PremierDraft,def6c2ea39ef4c51b6da6e470d911525,2023-11-09 20:56:46,bronze,7,1,2,8,Waylaying Pirates,0.7500,0.0,10,0.80,0.7500
1,LCI,PremierDraft,def6c2ea39ef4c51b6da6e470d911525,2023-11-09 20:56:46,bronze,7,1,2,13,Pirate Hat,0.2500,0.0,10,0.80,0.2500
2,LCI,PremierDraft,8e12fcabc9564699926dd155e483baf6,2023-11-09 18:16:27,bronze,4,3,0,6,Oltec Archaeologists,0.5713,0.0,100,0.64,0.5713
3,LCI,PremierDraft,8e12fcabc9564699926dd155e483baf6,2023-11-09 18:16:27,bronze,4,3,2,8,Helping Hand,0.4285,0.0,100,0.64,0.4285
4,LCI,PremierDraft,29b702ffe24144bfa7cbb922d20267fc,2023-11-09 18:06:30,bronze,1,3,0,5,Armored Kincaller,0.7500,0.0,100,0.50,0.7500
5,LCI,PremierDraft,29b702ffe24144bfa7cbb922d20267fc,2023-11-09 18:06:30,bronze,1,3,0,6,"Matzalantli, the Great Door",0.7500,0.0,100,0.50,0.7500
6,LCI,PremierDraft,29b702ffe24144bfa7cbb922d20267fc,2023-11-09 18:06:30,bronze,1,3,1,3,Tendril of the Mycotyrant,0.2500,0.0,100,0.50,0.2500
7,LCI,PremierDraft,29b702ffe24144bfa7cbb922d20267fc,2023-11-09 18:06:30,bronze,1,3,1,6,Deepfathom Echo,0.7500,0.0,100,0.50,0.7500
8,LCI,PremierDraft,29b702ffe24144bfa7cbb922d20267fc,2023-11-09 18:06:30,bronze,1,3,2,4,Self-Reflection,0.2500,0.0,100,0.50,0.2500
9,LCI,PremierDraft,29b702ffe24144bfa7cbb922d20267fc,2023-11-09 18:06:30,bronze,1,3,2,5,Poison Dart Frog,0.7500,0.0,100,0.50,0.7500
